# LDA - Latent Drichilet Allocation

In [4]:
#Dependencies
import pandas as pd
import gensim #the library for Topic modelling
from gensim.models.ldamulticore import LdaMulticore
from gensim import corpora, models
import pyLDAvis.gensim #LDA visualization library
import nltk
from wordcloud import WordCloud

from nltk.corpus import stopwords
import string
from nltk.stem.wordnet import WordNetLemmatizer

import warnings
warnings.simplefilter('ignore')
from itertools import chain

Read reviews from csv file

In [5]:
import pandas as pd
df = pd.read_excel(r'E:\Topic_Modeling\New_data\all\heated-jacket.xlsx');#, error_bad_lines=False);
df.head(10)

,product,title,rating,Message
0,Amazon.com: Customer reviews: Heated Vest for ...,Body warmer less frigid,4,I bought this for work related since I work in...
1,Amazon.com: Customer reviews: Heated Vest for ...,Takes off the edge of cold winter days,5,This is my first heated piece of clothing and ...
2,Amazon.com: Customer reviews: Heated Vest for ...,Nice and even heat without bulkiness.,5,The vest fits great. It is thin enough to not ...
3,Amazon.com: Customer reviews: Heated Vest for ...,I have a bad back,5,Always looking for other options for my arthri...
4,Amazon.com: Customer reviews: Heated Vest for ...,"Stylish, comfortable and warm!",5,I really love this vest! The battery lasts lon...
5,Amazon.com: Customer reviews: Heated Vest for ...,Quality and comfy material ! Good fit !,5,"I really liked the material they used, it stre..."
6,Amazon.com: Customer reviews: Heated Vest for ...,Perfect for the cold!,5,I bought this item to wear under my hoodie and...
7,Amazon.com: Customer reviews: Heated Vest for ...,3xl fits like xl!,4,Vest works well says men or women definitely w...
8,Amazon.com: Customer reviews: Heated Vest for ...,Very Well Designed!,5,The material is very soft. Very well designed....
9,Amazon.com: Customer reviews: Heated Vest for ...,Amazing Heated Jacket,5,This is exactly what I needed! I love having a...


Clean the data

In [7]:
#clean the data
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()

def clean(text):
    stop_free = ' '.join([word for word in text.lower().split() if word not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = ' '.join([lemma.lemmatize(word) for word in punc_free.split()])
    return normalized.split()

In [8]:
nltk.download('wordnet')
df['text_clean'] = df['Message'].apply(clean)

[nltk_data] Downloading package wordnet to C:\Users\Nakib
[nltk_data]     Hasan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Dataframe after cleaning

In [9]:
df

,product,title,rating,Message,text_clean
0,Amazon.com: Customer reviews: Heated Vest for ...,Body warmer less frigid,4,I bought this for work related since I work in...,"[bought, work, related, since, work, freezer, ..."
1,Amazon.com: Customer reviews: Heated Vest for ...,Takes off the edge of cold winter days,5,This is my first heated piece of clothing and ...,"[first, heated, piece, clothing, reminds, heat..."
2,Amazon.com: Customer reviews: Heated Vest for ...,Nice and even heat without bulkiness.,5,The vest fits great. It is thin enough to not ...,"[vest, fit, great, thin, enough, bulky, wear, ..."
3,Amazon.com: Customer reviews: Heated Vest for ...,I have a bad back,5,Always looking for other options for my arthri...,"[always, looking, option, arthritic, achy, bac..."
4,Amazon.com: Customer reviews: Heated Vest for ...,"Stylish, comfortable and warm!",5,I really love this vest! The battery lasts lon...,"[really, love, vest, battery, last, long, char..."
...,...,...,...,...,...
17064,B09G2KM6Q6,Good deal,4,Nothing,[nothing]
17065,B09G2KM6Q6,Great product! Highly recommend,5,"Very fast shipping, great product and customer...","[fast, shipping, great, product, customer, ser..."
17066,B09G2KM6Q6,C est un cadeau de Noël dure à dire.,4,Pour le beau-père qui a toujours froid.,"[pour, le, beaupère, qui, toujours, froid]"
17067,B09G2KM6Q6,warm for hours,5,staying warm on -3C in Canada,"[staying, warm, 3c, canada]"


Create Dictionary from the articles

In [10]:
#create dictionary
dictionary = corpora.Dictionary(df['text_clean'])
#Total number of non-zeroes in the BOW matrix (sum of the number of unique words per document over the entire corpus).
print(dictionary.num_nnz)

366768


In [11]:
print(df['text_clean'])

0        [bought, work, related, since, work, freezer, ...
1        [first, heated, piece, clothing, reminds, heat...
2        [vest, fit, great, thin, enough, bulky, wear, ...
3        [always, looking, option, arthritic, achy, bac...
4        [really, love, vest, battery, last, long, char...
                               ...                        
17064                                            [nothing]
17065    [fast, shipping, great, product, customer, ser...
17066           [pour, le, beaupère, qui, toujours, froid]
17067                          [staying, warm, 3c, canada]
17068                                    [tres, satisfait]
Name: text_clean, Length: 17069, dtype: object


In [12]:
'''
Create a dictionary from 'processed_docs' containing the number of times a word appears 
in the training set using gensim.corpora.Dictionary and call it 'dictionary'
'''
dictionary = gensim.corpora.Dictionary(df['text_clean'])

In [13]:
'''
Checking dictionary created
'''
count = 0

for k, v in dictionary.iteritems():
    
    print(k, v)
    
    count += 1
    
    if count > 10:
        break

0 also
1 another
2 around
3 bare
4 battery
5 better
6 body
7 bought
8 brand
9 bulky
10 burn


In [14]:
'''
OPTIONAL STEP
Remove very rare and very common words:

- words appearing less than 5 times
- words appearing in more than 50% of all documents
'''
# TODO: apply dictionary.filter_extremes() with the parameters mentioned above
dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n=100000)

In [15]:
'''
Create the Bag-of-words model for each document i.e for each document we create a dictionary reporting how many
words and how many times those words appear. Save this to 'bow_corpus'
'''
bow_corpus = [dictionary.doc2bow(doc) for doc in df['text_clean']]

In [16]:
print(bow_corpus)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Create document term matrix

In [18]:
#create document term matrix
doc_term_matrix = [dictionary.doc2bow(doc) for doc in df['text_clean'] ]
print(len(doc_term_matrix))

17069


# Instantiate LDA model

In [19]:
lda = gensim.models.ldamodel.LdaModel

Fit LDA model on the dataset

In [22]:
num_topics=30
%time ldamodel = lda(doc_term_matrix,num_topics=num_topics,id2word=dictionary,passes=50,minimum_probability=0)

Wall time: 5min 54s


Print the topics identified by LDA model

In [23]:
ldamodel.print_topics(num_topics=num_topics)

[(0,
  '0.299*"vest" + 0.075*"heated" + 0.039*"one" + 0.027*"much" + 0.019*"year" + 0.017*"bought" + 0.014*"ive" + 0.012*"winter" + 0.011*"golf" + 0.011*"im"'),
 (1,
  '0.099*"battery" + 0.060*"time" + 0.050*"long" + 0.043*"charge" + 0.039*"last" + 0.024*"work" + 0.023*"charged" + 0.022*"take" + 0.021*"day" + 0.019*"first"'),
 (2,
  '0.210*"best" + 0.066*"la" + 0.062*"ever" + 0.052*"mom" + 0.051*"le" + 0.039*"old" + 0.037*"de" + 0.037*"year" + 0.031*"e" + 0.023*"hockey"'),
 (3,
  '0.129*"wife" + 0.114*"gift" + 0.093*"christmas" + 0.071*"bought" + 0.061*"love" + 0.056*"cold" + 0.052*"it" + 0.045*"always" + 0.036*"loved" + 0.022*"present"'),
 (4,
  '0.032*"far" + 0.031*"outdoor" + 0.026*"perfect" + 0.026*"keeping" + 0.024*"job" + 0.023*"make" + 0.022*"anyone" + 0.018*"comfy" + 0.017*"boyfriend" + 0.015*"many"'),
 (5,
  '0.042*"wear" + 0.036*"fit" + 0.024*"layer" + 0.021*"shirt" + 0.017*"like" + 0.016*"little" + 0.015*"bit" + 0.015*"medium" + 0.015*"it" + 0.014*"i’m"'),
 (6,
  '0.132*"bat

Visualize the LDA model results

In [24]:
lda_display = pyLDAvis.gensim.prepare(ldamodel, doc_term_matrix, dictionary, sort_topics=False, mds='mmds')
pyLDAvis.display(lda_display)

Find which articles were marked in which cluster

In [25]:
# Assigns the topics to the documents in corpus
lda_corpus = ldamodel[doc_term_matrix]

In [26]:
[doc for doc in lda_corpus]

[[(0, 0.0004631662),
  (1, 0.19712986),
  (2, 0.0004631662),
  (3, 0.0004631662),
  (4, 0.0004631662),
  (5, 0.24566555),
  (6, 0.055128545),
  (7, 0.0004631662),
  (8, 0.033822455),
  (9, 0.0004631662),
  (10, 0.0004631662),
  (11, 0.0004631662),
  (12, 0.0004631662),
  (13, 0.0004631662),
  (14, 0.0004631662),
  (15, 0.20144369),
  (16, 0.0004631662),
  (17, 0.021506555),
  (18, 0.017948296),
  (19, 0.0004631662),
  (20, 0.05078343),
  (21, 0.0004631662),
  (22, 0.0004631662),
  (23, 0.04145746),
  (24, 0.0004631662),
  (25, 0.0004631662),
  (26, 0.0004631662),
  (27, 0.089028426),
  (28, 0.03728561),
  (29, 0.0004631662)],
 [(0, 0.10473688),
  (1, 0.17597908),
  (2, 0.17664215),
  (3, 0.0007578259),
  (4, 0.0007578259),
  (5, 0.0007578259),
  (6, 0.16612348),
  (7, 0.0007578259),
  (8, 0.0007578259),
  (9, 0.046796508),
  (10, 0.0007578259),
  (11, 0.0007578259),
  (12, 0.037358087),
  (13, 0.0007578259),
  (14, 0.0007578259),
  (15, 0.1603281),
  (16, 0.0007578259),
  (17, 0.023483

In [27]:
scores = list(chain(*[[score for topic_id,score in topic] \
                      for topic in [doc for doc in lda_corpus]]))

threshold = sum(scores)/len(scores)
print(threshold)

0.033333333386700796


In [31]:
cluster1 = [j for i,j in zip(lda_corpus,df.index) if i[0][1] > threshold]
cluster2 = [j for i,j in zip(lda_corpus,df.index) if i[1][1] > threshold]
cluster3 = [j for i,j in zip(lda_corpus,df.index) if i[2][1] > threshold]
cluster4 = [j for i,j in zip(lda_corpus,df.index) if i[3][1] > threshold]
cluster5 = [j for i,j in zip(lda_corpus,df.index) if i[4][1] > threshold]
cluster6 = [j for i,j in zip(lda_corpus,df.index) if i[5][1] > threshold]
cluster7 = [j for i,j in zip(lda_corpus,df.index) if i[6][1] > threshold]
cluster8 = [j for i,j in zip(lda_corpus,df.index) if i[7][1] > threshold]
cluster9 = [j for i,j in zip(lda_corpus,df.index) if i[8][1] > threshold]
cluster10 = [j for i,j in zip(lda_corpus,df.index) if i[9][1] > threshold]
cluster11 = [j for i,j in zip(lda_corpus,df.index) if i[10][1] > threshold]
cluster12 = [j for i,j in zip(lda_corpus,df.index) if i[11][1] > threshold]
cluster13 = [j for i,j in zip(lda_corpus,df.index) if i[12][1] > threshold]
cluster14 = [j for i,j in zip(lda_corpus,df.index) if i[13][1] > threshold]
cluster15 = [j for i,j in zip(lda_corpus,df.index) if i[14][1] > threshold]
cluster16 = [j for i,j in zip(lda_corpus,df.index) if i[15][1] > threshold]
cluster17 = [j for i,j in zip(lda_corpus,df.index) if i[16][1] > threshold]
cluster18 = [j for i,j in zip(lda_corpus,df.index) if i[17][1] > threshold]
cluster19 = [j for i,j in zip(lda_corpus,df.index) if i[18][1] > threshold]
cluster20 = [j for i,j in zip(lda_corpus,df.index) if i[19][1] > threshold]
cluster21 = [j for i,j in zip(lda_corpus,df.index) if i[20][1] > threshold]
cluster22 = [j for i,j in zip(lda_corpus,df.index) if i[21][1] > threshold]

print(len(cluster1))
print(len(cluster2))
print(len(cluster3))
print(len(cluster4))
print(len(cluster5))
print(len(cluster6))
print(len(cluster7))
print(len(cluster8))
print(len(cluster9))
print(len(cluster10))
print(len(cluster11))
print(len(cluster12))
print(len(cluster13))
print(len(cluster14))
print(len(cluster15))
print(len(cluster16))
print(len(cluster17))
print(len(cluster18))
print(len(cluster19))
print(len(cluster20))
print(len(cluster21))
print(len(cluster22))

3567
3778
1607
2689
2471
3674
3887
3905
2442
2924
2599
2890
1608
1827
3858
4322
1628
2471
3322
2605
3900
3624
